# 02. Preprocesamiento y Feature Engineering - AeroSafe Risk Predictor

**Objetivo:** Limpiar el dataset de Bird Strikes, manejar valores nulos, codificar variables categóricas y preparar el dataframe final para el entrenamiento de modelos.

In [1]:
import pandas as pd
import numpy as np
import os

# 1. Carga de datos 
data_path = r"D:\UTP\Empresa Aeronáutica\aerosafe-risk-predictor\data\raw\Bird_strikes.csv"
df = pd.read_csv(data_path, encoding="latin1")

# 2. Selección de columnas relevantes para el modelo de riesgo

cols_to_keep = [
    'AircraftType', 'AirportName', 'AltitudeBin', 'MakeModel', 
    'WildlifeSize', 'ConditionsSky', 'ConditionsPrecipitation',
    'Damage'  # Nuestra variable objetivo
]

df_clean = df[cols_to_keep].copy()

# 3. Manejo de Nulos inicial
print("Nulos por columna antes de limpiar:")
print(df_clean.isna().sum())


df_clean = df_clean.fillna('Unknown')

print("\nForma del dataset limpio:", df_clean.shape)
df_clean.head()

Nulos por columna antes de limpiar:
AircraftType                   0
AirportName                    0
AltitudeBin                    0
MakeModel                      0
WildlifeSize                   0
ConditionsSky                  0
ConditionsPrecipitation    23414
Damage                         0
dtype: int64

Forma del dataset limpio: (25429, 8)


,AircraftType,AirportName,AltitudeBin,MakeModel,WildlifeSize,ConditionsSky,ConditionsPrecipitation,Damage
0,Airplane,LAGUARDIA NY,"(1000, 2000]",B-737-400,Medium,No Cloud,Unknown,Caused damage
1,Airplane,DALLAS/FORT WORTH INTL ARPT,"(-1, 0]",MD-80,Small,Some Cloud,Unknown,Caused damage
2,Airplane,LAKEFRONT AIRPORT,"(30, 50]",C-500,Small,No Cloud,Unknown,No damage
3,Airplane,SEATTLE-TACOMA INTL,"(30, 50]",B-737-400,Small,Some Cloud,Unknown,No damage
4,Airplane,NORFOLK INTL,"(30, 50]",CL-RJ100/200,Small,No Cloud,Unknown,No damage


In [2]:
# 1. Variable objetivo binaria
df_model = df_clean.copy()

df_model["target_damage"] = (df_model["Damage"] == "Caused damage").astype(int)

print(df_model["target_damage"].value_counts())
df_model[["Damage", "target_damage"]].head()

target_damage
0    22975
1     2454
Name: count, dtype: int64


,Damage,target_damage
0,Caused damage,1
1,Caused damage,1
2,No damage,0
3,No damage,0
4,No damage,0


In [3]:
# 2. Separar variables predictoras y objetivo
X = df_model.drop(columns=["Damage", "target_damage"])
y = df_model["target_damage"]

X.head()

,AircraftType,AirportName,AltitudeBin,MakeModel,WildlifeSize,ConditionsSky,ConditionsPrecipitation
0,Airplane,LAGUARDIA NY,"(1000, 2000]",B-737-400,Medium,No Cloud,Unknown
1,Airplane,DALLAS/FORT WORTH INTL ARPT,"(-1, 0]",MD-80,Small,Some Cloud,Unknown
2,Airplane,LAKEFRONT AIRPORT,"(30, 50]",C-500,Small,No Cloud,Unknown
3,Airplane,SEATTLE-TACOMA INTL,"(30, 50]",B-737-400,Small,Some Cloud,Unknown
4,Airplane,NORFOLK INTL,"(30, 50]",CL-RJ100/200,Small,No Cloud,Unknown


In [5]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

# 3. Dividir en train y test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# 4. Definir columnas categóricas
cat_cols = X.columns.tolist()

# 5. One-Hot Encoder
preprocessor = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols)
    ]
)

# Ajustar en train y transformar train/test
X_train_enc = preprocessor.fit_transform(X_train)
X_test_enc = preprocessor.transform(X_test)

X_train_enc.shape, X_test_enc.shape

((20343, 1360), (5086, 1360))

In [6]:
import joblib
import os

# Definir ruta de guardado
processed_dir = r"D:\UTP\Empresa Aeronáutica\aerosafe-risk-predictor\data\processed"
os.makedirs(processed_dir, exist_ok=True)

# Guardar el preprocesador (para usarlo en el futuro con datos nuevos)
joblib.dump(preprocessor, os.path.join(processed_dir, "preprocessor_ohe.pkl"))

# Guardar los datos de entrenamiento y prueba
joblib.dump((X_train_enc, X_test_enc, y_train, y_test),
            os.path.join(processed_dir, "model_data.pkl"))

print("¡Éxito! Datos guardados en data/processed/")

¡Éxito! Datos guardados en data/processed/


In [1]:
# CELDA 1: Imports básicos
import pandas as pd
import numpy as np
import os

# Configuración opcional de pandas
pd.set_option("display.max_columns", 100)
pd.set_option("display.width", 120)

In [4]:
# CELDA 2: Cargar datos crudos usando ruta absoluta
import os

# Usamos la ruta exacta que confirmaste
data_raw_path = r"D:\UTP\AeroCVer\aerosafe-risk-predictor\data\raw\Bird_strikes.csv"

if os.path.exists(data_raw_path):
    df_raw = pd.read_csv(data_raw_path)
    print("Archivo cargado exitosamente.")
    display(df_raw.head())
else:
    print(f"Error: No se encontró el archivo en: {data_raw_path}")
    print("Verifica que el nombre de la carpeta sea AeroCVer o aerosafe-risk-predictor")

Archivo cargado exitosamente.


,RecordID,AircraftType,AirportName,AltitudeBin,MakeModel,NumberStruck,NumberStruckActual,Effect,FlightDate,Damage,Engines,Operator,OriginState,FlightPhase,ConditionsPrecipitation,RemainsCollected?,RemainsSentToSmithsonian,Remarks,WildlifeSize,ConditionsSky,WildlifeSpecies,PilotWarned,Cost,Altitude,PeopleInjured,IsAircraftLarge?
0,202152,Airplane,LAGUARDIA NY,"(1000, 2000]",B-737-400,Over 100,859,Engine Shut Down,11/23/00 0:00,Caused damage,2,US AIRWAYS*,New York,Climb,NaN,False,False,FLT 753. PILOT REPTD A HUNDRED BIRDS ON UNKN T...,Medium,No Cloud,Unknown bird - medium,N,"30,736",1500,0,Yes
1,208159,Airplane,DALLAS/FORT WORTH INTL ARPT,"(-1, 0]",MD-80,Over 100,424,NaN,7/25/01 0:00,Caused damage,2,AMERICAN AIRLINES,Texas,Landing Roll,NaN,False,False,102 CARCASSES FOUND. 1 LDG LIGHT ON NOSE GEAR ...,Small,Some Cloud,Rock pigeon,Y,0,0,0,No
2,207601,Airplane,LAKEFRONT AIRPORT,"(30, 50]",C-500,Over 100,261,NaN,9/14/01 0:00,No damage,2,BUSINESS,Louisiana,Approach,NaN,False,False,FLEW UNDER A VERY LARGE FLOCK OF BIRDS OVER AP...,Small,No Cloud,European starling,N,0,50,0,No
3,215953,Airplane,SEATTLE-TACOMA INTL,"(30, 50]",B-737-400,Over 100,806,Precautionary Landing,9/5/02 0:00,No damage,2,ALASKA AIRLINES,Washington,Climb,NaN,True,False,"NOTAM WARNING. 26 BIRDS HIT THE A/C, FORCING A...",Small,Some Cloud,European starling,Y,0,50,0,Yes
4,219878,Airplane,NORFOLK INTL,"(30, 50]",CL-RJ100/200,Over 100,942,NaN,6/23/03 0:00,No damage,2,COMAIR AIRLINES,Virginia,Approach,NaN,False,False,NO DMG REPTD.,Small,No Cloud,European starling,N,0,50,0,No


In [5]:
# CELDA 3: Análisis de valores nulos y tipos de datos
print("--- Valores Nulos por Columna ---")
print(df_raw.isnull().sum())

print("\n--- Tipos de Datos ---")
print(df_raw.dtypes)

--- Valores Nulos por Columna ---
RecordID                        0
AircraftType                    0
AirportName                     0
AltitudeBin                     0
MakeModel                       0
NumberStruck                    0
NumberStruckActual              0
Effect                      23351
FlightDate                      0
Damage                          0
Engines                       234
Operator                        0
OriginState                   449
FlightPhase                     0
ConditionsPrecipitation     23414
RemainsCollected?               0
RemainsSentToSmithsonian        0
Remarks                      4761
WildlifeSize                    0
ConditionsSky                   0
WildlifeSpecies                 0
PilotWarned                     0
Cost                            0
Altitude                        0
PeopleInjured                   0
IsAircraftLarge?                0
dtype: int64

--- Tipos de Datos ---
RecordID                     int64
AircraftTy

In [6]:
# CELDA 4: Explorar la variable Damage
print("Valores en la columna Damage:")
print(df_raw['Damage'].value_counts(dropna=False))

Valores en la columna Damage:
Damage
No damage        22975
Caused damage     2454
Name: count, dtype: int64


In [7]:
# CELDA 5: Transformación de Target y Fechas
df = df_raw.copy()

# 1. Crear variable objetivo binaria (1: Daño, 0: No daño)
df['IsSevere'] = df['Damage'].apply(lambda x: 1 if x == 'Caused damage' else 0)

# 2. Convertir FlightDate a datetime
df['FlightDate'] = pd.to_datetime(df['FlightDate'], errors='coerce')

# 3. Extraer Mes y Año (pueden ser factores de riesgo estacionales)
df['Month'] = df['FlightDate'].dt.month
df['Year'] = df['FlightDate'].dt.year

print("Variable 'IsSevere' creada y fechas transformadas.")
df[['FlightDate', 'Month', 'Year', 'Damage', 'IsSevere']].head()

C:\Users\Andre\AppData\Local\Temp\ipykernel_14676\151271763.py:8: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['FlightDate'] = pd.to_datetime(df['FlightDate'], errors='coerce')


Variable 'IsSevere' creada y fechas transformadas.


,FlightDate,Month,Year,Damage,IsSevere
0,2000-11-23,11,2000,Caused damage,1
1,2001-07-25,7,2001,Caused damage,1
2,2001-09-14,9,2001,No damage,0
3,2002-09-05,9,2002,No damage,0
4,2003-06-23,6,2003,No damage,0


In [8]:
# CELDA 6: Selección de columnas y limpieza de nulos
cols_to_keep = [
    'AircraftType', 'AirportName', 'AltitudeBin', 'MakeModel', 
    'WildlifeSize', 'ConditionsSky', 'PilotWarned', 'FlightPhase',
    'Month', 'Year', 'IsSevere'
]

df_clean = df[cols_to_keep].copy()

# Rellenar nulos en columnas categóricas con 'Unknown'
categorical_cols = df_clean.select_dtypes(include=['object']).columns
df_clean[categorical_cols] = df_clean[categorical_cols].fillna('Unknown')

# Eliminar filas donde el Mes o Año sean nulos (si los hay por error en fecha)
df_clean = df_clean.dropna(subset=['Month', 'Year'])

print(f"Dataset limpio listo. Forma final: {df_clean.shape}")
df_clean.isnull().sum()

Dataset limpio listo. Forma final: (25429, 11)


AircraftType     0
AirportName      0
AltitudeBin      0
MakeModel        0
WildlifeSize     0
ConditionsSky    0
PilotWarned      0
FlightPhase      0
Month            0
Year             0
IsSevere         0
dtype: int64

In [9]:
# CELDA 7: Guardar datos procesados
output_path = r"D:\UTP\AeroCVer\aerosafe-risk-predictor\data\processed\bird_strikes_clean.csv"

# Crear la carpeta si no existe (por seguridad)
os.makedirs(os.path.dirname(output_path), exist_ok=True)

df_clean.to_csv(output_path, index=False)
print(f"¡Éxito! Dataset guardado en: {output_path}")

¡Éxito! Dataset guardado en: D:\UTP\AeroCVer\aerosafe-risk-predictor\data\processed\bird_strikes_clean.csv


## 3. Resumen de Preprocesamiento

- Se creó la variable objetivo binaria `IsSevere` a partir de `Damage`.
- Se transformó `FlightDate` a tipo fecha y se generaron las columnas `Month` y `Year`.
- Se seleccionaron columnas relevantes (tipo de aeronave, aeropuerto, fase de vuelo, tamaño de fauna, etc.).
- Se imputaron valores faltantes categóricos con `'Unknown'`.
- Se guardó el dataset procesado en `data/processed/bird_strikes_clean.csv` para usarlo en los modelos.